# Customizing EAGO to Solve a Quasiconvex Problem 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

In the following example, we illustrate how EAGO's basic branch and bound routine can be adapted for use in other algorithms.

## An Algorithm for Solving Quasiconvex Problems

We'll adapt EAGO to implement the bisection based algorithm used to solve quasiconvex optimization problem presented in [1] to solve the below problem:

$
\begin{align}
   f^*= \qquad&\min_{\mathbf y\in Y} f(\mathbf y) \\ 
   {\rm s.t.}\;\;&\sum_{i=1}^5 i \cdot y_i - 5 = 0  \label{cons:first_qc} \\
   &\sum_{i=1}^5 y_i^2 - 0.5\pi \leq 0   \\
   &-\left(\frac{1}{2}y_1^2 + \frac{1}{2}y_2^2 + y_3^2 + 2y_1y_2 + 4y_1y_3 + 2y_2y_3\right)\leq 0   \\ 
   &-y_1^2 - 6y_1y_2 - 2y_2^2 + \text{cos}(y_1) + \pi \leq 0 \label{cons:last_qc} \\
    &Y = [0, 5]^5
\end{align}
$

where

$
\begin{align}
    f(\mathbf y) = -\frac{\text{ln}((5+y_1)^2 + \sum_{i=1}^5 y_i)}{1 + \sum_{i=1}^{5}y_{i}^{2}}.
\end{align}
$

Interval analysis shows that the objective value is bounded by the interval F such that $f^* \in F=[f^L, f^U] = [-5, 0]$. Introducing an auxiliary variable $t\in T=F$ allows the problem to be formulated as:

$
\begin{align}
    t^*=&\min_{\mathbf y\in Y,t\in T}t\\
    {\rm s.t.} \quad & (24)-(27)\\
    &f(\mathbf y)-t\le 0\\
    &Y=[0,5]^2,\;\;T=[-5,0].
\end{align}
$

Let $\phi_\tau(\mathbf y) = f(\mathbf y) - \tau$ such that $\tau = (t^L + t^U)/2$. We solve for $\mathbf y$ subject to constraints (24)-(27) were $\phi_\tau(\mathbf y) \leq 0$. If this is feasible, $t^* \in [t^L, \tau]$, else $t^* \in [\tau, t^U]$. The other interval containg $t^*$ is kept and the other is fathomed. This manner of bisection is repeated until an interval containing a feasible solution with a width of at most $\epsilon$ is located [2].

## Customizing EAGO's script
First, the preprocessing step, upper problem, and postprocessing routines are short circuited as only a single optimization problem need be solved at each iteration.

In [6]:
using MathOptInterface, EAGO, JuMP
import EAGO: Optimizer

struct QuasiConvex <: EAGO.ExtensionType end
import EAGO: preprocess!, upper_problem!, postprocess!
function EAGO.preprocess!(t::QuasiConvex, x::Optimizer)
    x._preprocess_feasibility = true
    end
function EAGO.upper_problem!(t::QuasiConvex, x::Optimizer)
    x._upper_feasibility = true
    end
function EAGO.postprocess!(t::QuasiConvex, x::Optimizer)
    x._postprocess_feasibility = true
end

Next we specify that only an absolute tolerance should be checked for convergence and termination.

In [7]:
import EAGO: convergence_check, termination_check, repeat_check
function EAGO.convergence_check(t::QuasiConvex, x::Optimizer)
    gap = (x._upper_objective_value - x._lower_objective_value)
    return (gap <= x.absolute_tolerance)
end
function EAGO.termination_check(t::QuasiConvex, x::Optimizer)
    flag = EAGO.convergence_check(t, x)
    if flag
        x._termination_status_code = MathOptInterface.OPTIMAL
        x._result_status_code = MathOptInterface.FEASIBLE_POINT
    end
    return flag
end

Only these sixth variable representing $t$ need be branched on.

In [8]:
branch_variable = [i == 6 for i=1:6]
EAGO.repeat_check(t::QuasiConvex, x::Optimizer) = true

In the lower problem, we then specify that the problem is solved locally for a fixed $t$ value. The objective value is 
updated and the problem is contracted in order to discard the region which is known not to contain the optimal value.

In [9]:
import EAGO: lower_problem!
function EAGO.lower_problem!(t::QuasiConvex, x::Optimizer)
    y = x._current_node
    lower = y.lower_variable_bounds[6]
    upper = y.upper_variable_bounds[6]
    midy = (lower + upper)/2.0
    y.lower_variable_bounds[6] = midy
    y.upper_variable_bounds[6] = midy
    EAGO.solve_local_nlp!(x)
    feas = x._upper_feasibility
    y.lower_variable_bounds[6] = feas ?  lower : midy
    y.upper_variable_bounds[6] = feas ?  midy : upper
    x._lower_objective_value = y.lower_variable_bounds[6]
    x._upper_objective_value = y.upper_variable_bounds[6]
    x._lower_feasibility = true
    return
end

We now build the JuMP model representing this problem, solve it and then retrieve the solution.

In [10]:
opt = with_optimizer(Optimizer, absolute_tolerance = 1E-8, branch_variable = branch_variable, ext_type = QuasiConvex())
m = Model(opt)
@variable(m, ((i<6) ? 0 : -5) <= y[i=1:6] <= ((i<6) ? 5 : 0))
@constraint(m, sum(i*y[i] for i=1:5) - 5 == 0)
@constraint(m, sum(y[i]^2 for i=1:5) - 0.5*pi^2 <= 0)
@expression(m, expr1, 2*y[1]*y[2] + 4*y[1]*y[3] + 2*y[2]*y[3])
@constraint(m, -(0.5*y[1]^2 + 0.5*y[2]^2 + y[3]^2 + expr1) <= 0)
@NLexpression(m, expr2, log((5 + y[1])^2 + sum(y[i] for i=1:5)))
@NLconstraint(m, -y[1]^2 -6*y[1]*y[2] -2*y[2]^2 +cos(y[1]) + pi <= 0)
@NLconstraint(m, -expr2/(1 + sum(y[i]^2 for i=1:5)) - y[6] <= 0)
@objective(m, Min, y[6])

JuMP.optimize!(m)# retrieve solution info
solution = JuMP.value.(y[1:5])
global_obj_value = JuMP.value.(y[6])

-------------------------------------------------------------------------------------------------------
|  Iteration #  |   Nodes  | Lower Bound | Upper Bound  |    Gap    |   Ratio   |  Time   | Time Left |
-------------------------------------------------------------------------------------------------------
First Solution Found at Node 1
UBD = -1.71690346673131
Solution is :
    X[1] = 0.6521533635051513
    X[2] = 0.6687269925064182
    X[3] = 0.1827815731700123
    X[4] = 0.24121532455559402
    X[5] = 0.2994373267499198
    X[6] = -1.71690346673131


-1.71690346673131

### Reference:
1. C. Jansson,Quasiconvex relaxations based on interval arithmetic, Linear Algebra andits Applications, 324 (2001), pp. 27–53.
2. S. Boyd and L. Vandenberghe,Convex optimization, Cambridge university press,2004.